In [20]:
# Import the necessary libraries
import pandas as pd
import numpy as np

In [21]:
# Loading 'cartwheelProyect.xlsx' Excel file using pandas and assigning it to 'cartwheel' dataframe
cartwheel = pd.read_excel('CartwheelProyect.xlsx')

In [22]:
# Convert date columns from object to datetime datatype
date_columns = ['Order Created', 'Actual Drop off Time', 
                'Actual Pick up Time', 'Finish Date']
for col in date_columns:
    cartwheel[col] = pd.to_datetime(cartwheel[col], errors='coerce')

In [23]:
# Drop rows with missing values in these columns
cartwheel.dropna(subset=['Order Created', 'Finish Date', 
                         'Actual Drop off Time', 'Actual Pick up Time'], 
                inplace=True)

In [24]:
# Calculate delivery time metric
cartwheel['Delivery Time'] = (cartwheel['Actual Drop off Time'] - 
                              cartwheel['Order Created']).dt.total_seconds() / 3600

In [25]:
# Calculate time between pick-up and drop-off
cartwheel['Pick up to Drop off Time'] = (cartwheel['Actual Drop off Time'] - 
                                         cartwheel['Actual Pick up Time']).dt.total_seconds() / 3600


In [26]:
# NOTE: 'Subtotal' column has too many missing values, so I will not include it in my analysis for integrity of the analysis.

In [46]:
# Convert 'Tip' column to numeric, coerce errors to NaN
cartwheel['Tip'] = pd.to_numeric(cartwheel['Tip'], errors='coerce')

# Fill missing values in 'Tip' with 0 assuming no tip was given for these entries
cartwheel['Tip'].fillna(0, inplace=True)

In [28]:
# NOTE: 'Total Revenue' calculation will be skipped because 'Subtotal' is not available for the big amount of missing values.

In [29]:
# Create a new column for total delivery time
cartwheel['Delivery Time'] = cartwheel['Finish Date'] - cartwheel['Order Created']

In [30]:
# Drop rows where 'Driver' is missing
cartwheel.dropna(subset=['Driver'], inplace=True)

In [31]:
# Analysis of driver efficiency
driver_efficiency = cartwheel.groupby('Driver')['Delivery Time'].mean()
print(driver_efficiency)

Driver
A B. [12130850]               2 days 08:56:30
AARON E. [21541484]           0 days 19:15:00
ADAM R. [21629178]            0 days 06:16:00
ADAN B. [17961455]            0 days 19:35:00
ALAN S. [21629525]            0 days 19:52:00
                                    ...      
nestor B. [17617988]          2 days 23:23:00
qui p. [21190159]             0 days 17:50:00
takaiszah s. [21010343]       1 days 02:32:00
Ã_x0081_lvaro R. [18307141]   0 days 23:30:00
Óscar R. [14465576]           0 days 22:25:00
Name: Delivery Time, Length: 1675, dtype: timedelta64[ns]


In [32]:
# Analysis of tip trends
cartwheel['Hour of Day'] = cartwheel['Order Created'].dt.hour
tip_trends = cartwheel.groupby('Hour of Day')['Tip'].mean()
print(tip_trends)

Hour of Day
0      9.813750
1     31.495000
2      3.454545
3      9.583333
4      9.155938
5     17.418148
6     21.038814
7     18.696538
8     23.671603
9     24.579609
10    22.785506
11    19.806967
12    12.636435
13    14.714734
14    15.694150
15    17.484733
16    14.297076
17    17.971767
18    22.619403
19    18.914661
20    17.151250
21    12.041531
22    18.268750
23    16.463043
Name: Tip, dtype: float64


In [33]:
# NOTE: 'Restaurant Delay' column has too many missing values, so I will not include it in my analysis.

In [34]:
# Temporal trends analysis
order_trends = cartwheel.groupby(cartwheel['Order Created'].dt.to_period("M"))['Order Number'].count()
print(order_trends)

Order Created
2022-06     630
2022-07     700
2022-08     872
2022-09     932
2022-10    1011
2022-11     787
2022-12    1680
2023-01    1108
2023-02     419
2023-03     485
2023-04     738
2023-05     979
2023-06       1
Freq: M, Name: Order Number, dtype: int64


In [48]:
# Ensure 'Hour of Day' is a valid hour
cartwheel = cartwheel.dropna(subset=['Hour of Day'])
cartwheel = cartwheel[cartwheel['Hour of Day'].between(0, 23)]  # assuming 'Hour of Day' is an integer representing the hour

# Select the 'Hour of Day' and 'Tip' columns and export to Excel
cartwheel[['Hour of Day', 'Tip']].to_excel('hourly_tips.xlsx', index=False)

In [37]:
# Select the relevant columns to build the Dashboard
columns_for_dashboard = ['Order Number', 'Order Status', 'Order Created', 'Finish Date', 'Actual Pick up Time', 'Actual Drop off Time', 
                         'Driver', 'Tip', 'Courier Charge', 'Delivery Time', 'Pick up to Drop off Time', 'Cancelled or Returned', 'Hour of Day']

dashboard_data = cartwheel[columns_for_dashboard]

# Save to Excel
dashboard_data.to_excel('DashboardData.xlsx', index=False)

In [38]:
# Number of orders per day
orders_per_day = cartwheel.groupby(cartwheel['Order Created'].dt.date)['Order Number'].count()
orders_per_day = orders_per_day.reset_index()
orders_per_day.columns = ['Date', 'Number of Orders']
orders_per_day.to_excel('orders_per_day.xlsx', index=False)
print(orders_per_day)
orders_per_day.to_excel('orders_per_day.xlsx', index=False)

# Number of orders per hour
orders_per_hour = cartwheel.groupby(cartwheel['Hour of Day'])['Order Number'].count()
orders_per_hour = orders_per_hour.reset_index()
orders_per_hour.columns = ['Hour of Day', 'Number of Orders']
orders_per_hour.to_excel('orders_per_hour.xlsx', index=False)
print(orders_per_hour)
orders_per_hour.to_excel('orders_per_hour.xlsx', index=False)

           Date  Number of Orders
0    2022-06-14                 1
1    2022-06-15                28
2    2022-06-16                61
3    2022-06-17                20
4    2022-06-18                 4
..          ...               ...
323  2023-05-28                 2
324  2023-05-29                37
325  2023-05-30                41
326  2023-05-31                41
327  2023-06-01                 1

[328 rows x 2 columns]
    Hour of Day  Number of Orders
0             0                 8
1             1                 4
2             2                11
3             3                48
4             4                64
5             5                81
6             6                59
7             7               104
8             8               343
9             9               640
10           10               494
11           11               821
12           12              1585
13           13              1821
14           14              1217
15           15         

In [45]:
# Delivery time by driver for export to Tableau.
# Export data to Excel without averaging
cartwheel[['Driver', 'Delivery Time']].to_excel('cartwheel_driver_data_driver_time.xlsx', index=False)


In [40]:
# Number of cancellations and returns by reason
cancellations_returns = cartwheel[cartwheel['Order Status'].isin(['Cancelled', 'Returned'])]
cancellations_returns_by_reason = cancellations_returns.groupby('Return Reason')['Order Number'].count()
cancellations_returns_by_reason = cancellations_returns_by_reason.reset_index()
cancellations_returns_by_reason.columns = ['Return Reason', 'Number of Orders']
cancellations_returns_by_reason.to_excel('cancellations_returns_by_reason.xlsx', index=False)

In [42]:
# Average tip by hour of day for export to Tableau.
# Export data to Excel without averaging
cartwheel[['Hour of Day', 'Tip']].to_excel('cartwheel_data_tip_hour.xlsx', index=False)